In [14]:
import autogen
from prompts import *
from auto_functions import CustomGroupChat, is_termination_msg
import random
from typing import List, Dict
from autogen.agentchat.groupchat import GroupChat
from autogen.agentchat.agent import Agent
from autogen.agentchat.assistant_agent import AssistantAgent

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",  # or OAI_CONFIG_LIST.json if file extension is added
    filter_dict={
        "model": {
            "gpt-4-1106-preview",
            "gpt-3.5-turbo-1106",
            "gpt-3.5-tubro",
            "gpt-4-vision-preview"
        }
    }
)

In [7]:
gpt_config = {
    "cache_seed": 42,
    "temperature": 0.4,
    "config_list":config_list,
    "timeout":120,
}

Human = autogen.UserProxyAgent(
    name = "Human_Student",
    system_message = human_prompt,
    code_execution_config=False,
    human_input_mode= 'ALWAYS'
)
question_generator = autogen.AssistantAgent(
    name = "AI_Teacher",
    llm_config=gpt_config,
    system_message= question_generator_prompt,
)

AI_student = autogen.AssistantAgent(
    name = "AI_student",
    llm_config=gpt_config,
    system_message= AI_student_prompt
)

AI_quality_control = autogen.AssistantAgent(
    name = "AI_Quality_Control",
    llm_config= gpt_config,
    system_message= AI_quality_control_prompt
)

AI_grader = autogen.AssistantAgent(
    name = "Grader",
    llm_config=gpt_config,
    system_message = grader_prompt,
)

theory_discusser = autogen.AssistantAgent(
    name = "Theory_Discusser",
    llm_config=gpt_config,
    system_message = theory_discusser_prompt)

clarifying_AI = autogen.AssistantAgent(
    name = "Question_Clarifier",
    llm_config=gpt_config,
    system_message = clarifying_AI_prompt)

groupchat = autogen.GroupChat(agents = [Human, question_generator, AI_student, AI_quality_control, AI_grader, theory_discusser, clarifying_AI],messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)

In [8]:
Human.initiate_chat(
    manager,
    message=groupcat_initial_prompt,
)

Human_Student (to chat_manager):


Your goal is to work together to create a question for a human student to answer, to grade that answer, and to provide feedback as necessary.

The general flow of the conversation should be as follows:

1. AI_Teacher should generate a question based on the provided information.
The question should be sent to the human student for them to answer.
2. While the human student is answering the question, AI student answers the question before the human student
3. AI quality control checks that the answer the AI student gave is correct and of good quality
4. wait for the Human student to provide an answer
5. Grader will take the human answer and grade it
6. Depending on the result of the grade, the Human student will either be congratulated or asked to try again if they got it wrong (less then 80%) or they the grader will tell the student they are incorrect and provide the solution if they got it wrong three times.
7. If the student wants to discuss theory w

KeyboardInterrupt: 